In [1]:
from pathlib import Path

from ray import tune
from ray.air import Checkpoint, session

In [2]:
ray_results = Path("~/ray_results").expanduser()
assert ray_results.is_dir()
last_experiment = max(ray_results.glob('_aux_train_daostack_*'), key=lambda x: x.stat().st_ctime)
print(f"Loading results from {last_experiment}...")
assert last_experiment.is_dir()

def _aux_train_daostack(*args, **kwargs):
    raise NotImplementedError

# TODO: Create an experiment grid or whatever is called
tuner = tune.Tuner.restore(str(last_experiment), _aux_train_daostack)
rg = tuner.get_results()
rg.get_best_result()

Loading results from /home/davo/ray_results/_aux_train_daostack_2023-08-16_20-34-00...


2023-08-16 20:36:57,453	INFO experiment_analysis.py:972 -- No trial data passed in during `ExperimentAnalysis` initialization -- you are most likely loading the experiment after it has completed.
Loading trial data from the experiment checkpoint file. This may result in loading some stale information, since checkpointing is periodic.
2023-08-16 20:36:57,466	WARNING experiment_analysis.py:916 -- Failed to read the results for 1 trials:
- /home/davo/ray_results/_aux_train_daostack_2023-08-16_20-34-00/_aux_train_daostack_59dd3f68_3_trial_index=2,batch_size=6,conv_layers=3,embedding_dim=230,l2=0.0000,learning_rate=0.0001,max_epoch_2023-08-16_20-34-15


Result(
  metrics={'loss': 0.43998135564844854, 'rprec train': 0.3339490592479706, 'rprec test': 0.3339490592479706, 'p@5 train': 0.42499998211860657, 'p@5 test': 0.42499998211860657, 'r@5 train': 0.08189654350280762, 'r@5 test': 0.08189654350280762, 'should_checkpoint': True, 'done': False, 'trial_id': 'e7978e0d', 'experiment_tag': '1_trial_index=0,batch_size=6,conv_layers=3,embedding_dim=230,l2=0.0000,learning_rate=0.0001,max_epochs=50'},
  path='/home/davo/ray_results/_aux_train_daostack_2023-08-16_20-34-00/_aux_train_daostack_e7978e0d_1_trial_index=0,batch_size=6,conv_layers=3,embedding_dim=230,l2=0.0000,learning_rate=0.0001,max_epoch_2023-08-16_20-34-04',
  checkpoint=Checkpoint(local_path=/home/davo/ray_results/_aux_train_daostack_2023-08-16_20-34-00/_aux_train_daostack_e7978e0d_1_trial_index=0,batch_size=6,conv_layers=3,embedding_dim=230,l2=0.0000,learning_rate=0.0001,max_epoch_2023-08-16_20-34-04/checkpoint_000010)
)

In [3]:
# Read dataframe and drop some unneeded columns
rdf = rg.get_dataframe('p@5 test', 'max').drop(columns=['timestamp', 'node_ip', 'pid', 'hostname', 'time_since_restore', 'iterations_since_restore', 'logdir'])
rdf = rdf.drop(columns=['date'])
# rdf = rdf[rdf['done']]
print(rdf.columns)

for c in ['config/batch_size']: #, 'config/embedding_dim']:
    rdf[c] = 2**rdf[c]

rdf.sort_values('loss').head(5)

2023-08-16 20:36:57,494	WARNING experiment_analysis.py:694 -- Failed to read the config for 1 trials:
- /home/davo/ray_results/_aux_train_daostack_2023-08-16_20-34-00/_aux_train_daostack_59dd3f68_3_trial_index=2,batch_size=6,conv_layers=3,embedding_dim=230,l2=0.0000,learning_rate=0.0001,max_epoch_2023-08-16_20-34-15


Index(['loss', 'rprec train', 'rprec test', 'p@5 train', 'p@5 test',
       'r@5 train', 'r@5 test', 'time_this_iter_s', 'should_checkpoint',
       'done', 'training_iteration', 'trial_id', 'time_total_s',
       'config/__trial_index__', 'config/batch_size', 'config/conv_layers',
       'config/embedding_dim', 'config/l2', 'config/learning_rate',
       'config/max_epochs'],
      dtype='object')


,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,should_checkpoint,done,training_iteration,trial_id,time_total_s,config/__trial_index__,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,config/max_epochs
0,0.691052,0.722094,0.722094,0.951923,0.951923,0.264030,0.264030,14.996376,True,False,1,e7978e0d,14.996376,0,64,3,230,0.000003,0.00013,50
1,0.691081,0.709765,0.709765,0.955769,0.955769,0.264074,0.264074,15.669194,True,False,1,f8f3af67,15.669194,1,64,3,230,0.000003,0.00013,50


In [4]:
# Group by config (except trial index) and do the mean
non_config_columns = ['config/__trial_index__', 'config/max_epochs']
cfg_columns = [c for c in rdf.columns if c.startswith('config/') and c not in non_config_columns]
non_results_cols = ['should_checkpoint', 'done', 'config/__trial_index__'] + non_config_columns
rdfgb = rdf.drop(columns=non_results_cols).groupby(cfg_columns).mean(numeric_only=True).reset_index()
print(len(rdfgb), "configurations were tested")
rdfgb.sort_values('p@5 test', ascending=False).tail(30)

1 configurations were tested


,config/batch_size,config/conv_layers,config/embedding_dim,config/l2,config/learning_rate,loss,rprec train,rprec test,p@5 train,p@5 test,r@5 train,r@5 test,time_this_iter_s,training_iteration,time_total_s
0,64,3,230,0.000003,0.00013,0.691066,0.71593,0.71593,0.953846,0.953846,0.264052,0.264052,15.332785,1.0,15.332785
